In [45]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/debt-default-prediction/X_test.csv
/kaggle/input/debt-default-prediction/DataDictionary.xlsx
/kaggle/input/debt-default-prediction/valid.csv
/kaggle/input/debt-default-prediction/train.csv


# Train dataset is read and the rows with missing target value is removed from the dataset.

In [46]:
X_train = pd.read_csv('/kaggle/input/debt-default-prediction/train.csv')
X_train.dropna(axis=0, subset=['loan_status'], inplace=True)
X_train.head()
y_train = X_train.loan_status
X_train.drop(['loan_status'], axis = 1,inplace= True )


In [47]:
print(X_train.shape)
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column >0])

(517788, 144)
id                       517788
member_id                517788
emp_title                 34051
emp_length                31300
url                      517788
                          ...  
settlement_status        498528
settlement_date          498528
settlement_amount        498528
settlement_percentage    498528
settlement_term          498528
Length: 104, dtype: int64


In [48]:
num_cols_with_missing = (missing_val_count_by_column >50000).sum()
num_cols_with_missing

59

In [49]:
# Identify columns with more than 50,000 missing values
cols_to_drop = missing_val_count_by_column[missing_val_count_by_column > 50000].index

# Ensure that the columns to drop actually exist in X_train
cols_to_drop = [col for col in cols_to_drop if col in X_train.columns]

# Drop columns from X_train
X_train.drop(cols_to_drop, axis=1, inplace=True)

# Print the shape of X_train after dropping columns
print(X_train.shape)

(517788, 85)


In [50]:
X_valid = pd.read_csv('/kaggle/input/debt-default-prediction/valid.csv')
X_valid.dropna(axis=0, subset=['loan_status'], inplace=True)

y_valid = X_valid.loan_status
X_valid.drop(['loan_status'], axis = 1,inplace= True )
train_columns  = X_train.columns
X_valid = X_valid[train_columns]
X_valid.shape

(172596, 85)

* **Let us handle the rest of the missing values with imputations , numerical values with the mean of the column and the object columns with the most frequent value.**

In [51]:
unique_values_counts = X_train.nunique()

# Find columns where number of unique values is equal to 1
columns_with_same_value = unique_values_counts[unique_values_counts == 1].index.tolist()

# Print columns with the same value for every entry

X_train = X_train.drop(columns=columns_with_same_value)
X_valid = X_valid.drop(columns=columns_with_same_value)
print(X_train.shape, X_valid.shape)

(517788, 80) (172596, 80)


In [52]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train.columns if 
                    X_train[cname].dtype == 'object']

numerical_imputer = SimpleImputer(strategy='mean')
X_train_numerical = X_train[numerical_cols].copy()
X_valid_numerical = X_valid[numerical_cols].copy()
X_train_numerical = numerical_imputer.fit_transform(X_train_numerical)
X_valid_numerical = numerical_imputer.transform(X_valid_numerical)

# Preprocessing for categorical data
categorical_imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical = X_train[categorical_cols].copy()
X_valid_categorical = X_valid[categorical_cols].copy()
X_train_categorical = categorical_imputer.fit_transform(X_train_categorical)
X_valid_categorical = categorical_imputer.transform(X_valid_categorical)

# Convert back to DataFrame

X_train_numerical = pd.DataFrame(X_train_numerical, columns=numerical_cols)
X_train_categorical = pd.DataFrame(X_train_categorical, columns=categorical_cols)
X_valid_numerical = pd.DataFrame(X_valid_numerical, columns=numerical_cols)
X_valid_categorical = pd.DataFrame(X_valid_categorical, columns=categorical_cols)


# Now you can combine the numerical and categorical data
X_train = pd.concat([X_train_numerical, X_train_categorical], axis=1)
X_valid =  pd.concat([X_valid_numerical, X_valid_categorical], axis=1)


In [53]:
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column >0])
missing_val_count_by_column_valid = (X_valid.isnull().sum())
print(missing_val_count_by_column_valid[missing_val_count_by_column_valid >0])
print(X_train.shape, X_valid.shape)

Series([], dtype: int64)
Series([], dtype: int64)
(517788, 80) (172596, 80)


In [54]:
X_train[categorical_cols].head()

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,purpose,title,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,disbursement_method,debt_settlement_flag
0,36 months,A,A4,Paralegal,1 year,MORTGAGE,Not Verified,Aug-2017,debt_consolidation,Debt consolidation,740xx,OK,Feb-2003,f,Dec-2018,Dec-2018,Individual,Cash,N
1,36 months,D,D2,Teacher,10+ years,MORTGAGE,Verified,Jul-2014,debt_consolidation,Debt consolidation,337xx,FL,Mar-1982,w,Jul-2017,Jul-2017,Individual,Cash,N
2,36 months,A,A4,owner,4 years,MORTGAGE,Not Verified,Mar-2016,credit_card,Credit card refinancing,786xx,TX,Jul-1997,f,Oct-2017,Oct-2017,Individual,Cash,N
3,36 months,B,B5,Teacher,10+ years,MORTGAGE,Source Verified,Jan-2015,debt_consolidation,Debt consolidation,780xx,TX,Apr-1998,f,Jan-2018,Dec-2017,Individual,Cash,N
4,36 months,A,A4,Senior UX designer,< 1 year,MORTGAGE,Source Verified,Mar-2016,credit_card,Credit card refinancing,191xx,PA,Jan-2001,w,Dec-2017,Jul-2018,Individual,Cash,N


In [55]:
X_train['emp_length'].unique()

array(['1 year', '10+ years', '4 years', '< 1 year', '5 years', '2 years',
       '7 years', '9 years', '3 years', '8 years', '6 years'],
      dtype=object)

In [56]:
from sklearn.preprocessing import OrdinalEncoder
#custom_order_grade = ['A', 'B','C','D','E','F','G']
custom_order_subgrade = ['A1','A2','A3','A4','A5','B1','B2','B3','B4','B5','C1','C2','C3','C4','C5','D1','D2','D3','D4','D5','E1','E2','E3','E4','E5','F1','F2','F3','F4','F5','G1','G2','G3','G4','G5']
ordinal_encoder = OrdinalEncoder(categories=[custom_order_subgrade])
X_train['sub_grade'] = ordinal_encoder.fit_transform(X_train[['sub_grade']])


In [57]:
X_valid['sub_grade'] = ordinal_encoder.transform(X_valid[['sub_grade']]) 
custom_order_emp_length = ['< 1 year', '1 year', '2 years', '3 years','4 years',  '5 years','6 years', '7 years','8 years','9 years','10+ years']
ordinal_encoder2 = OrdinalEncoder(categories=[custom_order_emp_length])
X_train['emp_length'] = ordinal_encoder2.fit_transform(X_train[['emp_length']])
X_valid['emp_length'] = ordinal_encoder2.transform(X_valid[['emp_length']])
print('done')

done


In [61]:
X_valid['sub_grade'].head()

0    16.0
1    11.0
2    11.0
3     6.0
4     8.0
Name: sub_grade, dtype: float64

In [63]:
X_train.drop(columns = ['grade'], inplace=True)
X_valid.drop(columns = ['grade'], inplace=True) # redundant feature with grade
